In [28]:
from pathlib import Path
import topagnps2cche1d as t2c

# from t2c import Watershed, CrossSection
# from topagnps2cche1d import Watershed, CrossSection

casename = "goodwin_creek_1m_3_Reach_Station_12"

root_dir = Path(f"../data_inputs/{casename}/TopAGNPS_DataSets/3_Reach")
# root_dir = Path("../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach")
# root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"

path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"

step = 30
strahler = 0

In [29]:
watershed = t2c.Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.import_topagnps_cells(path_to_cells)

watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)

watershed.assign_strahler_number_to_reaches()

Reading Reaches: 100%|██████████| 3/3 [00:00<00:00, 80.57it/s]


In [30]:
watershed.ignore_reaches_with_strahler_leq(strahler)


# watershed.reaches[4].ignore_reach()

# watershed.update_graph()
# watershed.update_junctions_and_node_types()
# watershed.identify_inflow_sources()

# watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
# watershed.set_node_id_to_compute_id()

watershed.update_watershed()

watershed.resample_reaches(id_list="all", step=step)

watershed.assign_cross_section_to_all_points()
watershed.adjust_cross_sections_elevation_with_slope()

In [31]:
# watershed.resample_reaches(id_list="all", step=50)

# watershed.assign_cross_section_to_all_points()

In [32]:
df_nodes = watershed.create_cche1d_nodes_df()
df_ch = watershed.create_cche1d_channels_df()
df_lk, df_rc = watershed.create_cche1d_links_and_reaches_df()
df_csec, df_csprf = watershed.create_cche1d_csec_csprf_df()
df_tw = watershed.create_cche1d_twcells_df()
df_inflow_reaches = watershed.get_list_of_inflow_reaches_df()

In [33]:
casename_strahler_step = f"{casename}_strahler_gt{strahler}_step_{step}m"

# step = "native_"
# strahler = 0
# casename = f"ohio_strahler_gt{strahler}_step_{step}m"
watershed.write_cche1d_dat_files(
    casename=casename_strahler_step,
    output_folder=f"./../data_outputs/{casename_strahler_step}",
)

## Plotting network

In [34]:
watershed.plot(
    frame_height=700, frame_width=900, line_width=2, by="CCHE1D_ID", title="Watershed"
)

:Overlay
   .NdOverlay.I  :NdOverlay   [CCHE1D_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [CCHE1D_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)